In [1]:
import sys
sys.path.insert(1,'../tools')
from transformers import modeling_bert
import numpy as np
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from tqdm import tqdm
import pdb
import torch.nn.functional as F
from torch import optim
import time
import utils
import imp
#from utils import batch_sent_loader
import os
from sklearn.metrics import confusion_matrix
import json
import sklearn

In [2]:
imp.reload(utils)
hpara1 = utils.hpara()
hpara1.ratio = 2
hpara1.batch_size = 4
hpara1.word_layers=4
hpara1.sent_layers=1
hpara1.use_position_embedding = False
hpara1.word_lr = 1e-04 #5e-05
hpara1.sent_lr = 8e-04 #2e-04
hpara1.decay_step = 10
hpara1.decay_gamma = 0.5
hpara1.max_sent_len = 64
hpara1.fix_cls = True
hpara1.head_num = 12
hpara1.att_decay_step=3
hpara1.att_decay_rate=0.5
hpara1.max_doc_len = 300

In [3]:
hpara1.use_SSI_Bert = False
hpara1.use_narrow = False

In [4]:
if hpara1.use_SSI_Bert:
    pretrain_model_dir = '../BERT/bert/infection_bert_fix_gast'
else:
    pretrain_model_dir = '../BERT/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000'
vocab_file = os.path.join(pretrain_model_dir,'vocab.txt')
bert_config_file = os.path.join(pretrain_model_dir,'bert_config.json')

In [5]:
trained_word_model = './exp/best_clinical_bert/save_word_10.bin' 
trained_sent_model = './exp/best_clinical_bert/save_sent_10.bin' 

if hpara1.use_SSI_Bert and 'SSI' not in trained_word_model:
    print('ERROR')
if not hpara1.use_SSI_Bert and 'SSI'  in trained_word_model:
    print('ERROR')

In [6]:
imp.reload(utils)
all_generator = utils.load_data_for_correction()
tokenizer = utils._load_tf_tokenizer(vocab_file = vocab_file)

2887



In [7]:
config = modeling_bert.BertConfig.from_json_file(bert_config_file)
config.num_hidden_layers = hpara1.word_layers
config.output_attentions = True
model_word = modeling_bert.BertModel(config)
model_word.load_state_dict(torch.load(trained_word_model))
cls_weight = model_word.state_dict()['embeddings.word_embeddings.weight'][101]

In [8]:
config_doc = modeling_bert.BertConfig.from_json_file(bert_config_file)
config_doc.num_hidden_layers = hpara1.sent_layers
config_doc.output_attentions = True
config_doc.num_attention_heads = hpara1.head_num
use_position_embedding = hpara1.use_position_embedding
model_sent = modeling_bert.BertModel_no_embedding(config_doc,cls_weight,use_position_embedding=use_position_embedding)
model_sent.load_state_dict(torch.load(trained_sent_model))

<All keys matched successfully>

In [9]:
model_word = model_word.cuda()
model_sent = model_sent.cuda()

In [10]:
date = str(list(time.localtime())[0:3]).replace(', ','_')
save_dir = './exp/label_correction'+'_'+date[1:-1]+'/'

    
if not os.path.exists(save_dir):
    cmd = 'mkdir -p ' + save_dir
    os.system(cmd)
     
with open(save_dir + 'used_model.txt','w') as f:
    f.writelines(trained_word_model + '\n' +trained_sent_model)
    
with open(save_dir + 'hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)
    
max_epoch = hpara1.max_epoch
log = 'Iter {}/{}, Loss={:.3f},accu={:.3f},time={:.3f}\n'
from tqdm import tqdm
batch_size = hpara1.batch_size
accumulation_steps = hpara1.accumulation_steps
max_sent_len = hpara1.max_sent_len
max_doc_len = hpara1.max_doc_len
#progress_bar = tqdm(enumerate(training_generator))
para_dict = {}
hpara_list = []

In [11]:
type(all_generator)

torch.utils.data.dataloader.DataLoader

In [12]:
for data in all_generator:
    pass

In [ ]:
start = time.time()
model_sent.eval()
model_word.eval()
pred_list = []
y_list = []
y_hat = []
total_num = 0
correct = 0
max_doc_len = hpara1.max_doc_len
batch_size = hpara1.batch_size
narrow = hpara1.use_narrow
for doc_count,(doc,label) in enumerate(all_generator):
    total_num += len(label)
    label = label.cuda()
    batch_count=0
    sent_num = 0
    end_ind = 0
    input_tensors = torch.zeros([1,max_doc_len,hpara1.hidden_size]).cuda()
    while end_ind <= len(doc) and end_ind < max_doc_len-1:
        batch_sent = utils.batch_sent_loader(doc,batch_size,batch_count,max_doc_len=max_doc_len)
        cur_batch_size = len(batch_sent)
        sent_num += cur_batch_size
        input_ids = torch.zeros(cur_batch_size,max_sent_len).long().cuda()
        input_mask = torch.ones(cur_batch_size,max_sent_len).long().cuda()
        for i in range(len(batch_sent)):
            tmp_ids,tmp_mask= utils.word_tokenize(batch_sent[i][0],max_sent_len,tokenizer)
            input_ids[i,:] = torch.tensor(tmp_ids)
            input_mask[i,:] = torch.tensor(tmp_mask)
        _,tmp_input_tensors,word_att_output = model_word(input_ids,attention_mask=input_mask)
        start_ind = batch_count*batch_size+1
        end_ind = start_ind + cur_batch_size
        input_tensors[0,start_ind:end_ind] = tmp_input_tensors
        batch_count += 1

    sent_mask = [1]*(end_ind)
    while len(sent_mask)<max_doc_len:
        sent_mask.append(0)
    sent_mask = torch.tensor(sent_mask).unsqueeze(0).cuda()
    if narrow:
        _,proba,sent_att_output = model_sent(input_tensors,label,attention_mask=sent_mask,epoch=cur_epoch)
    else:
        _,proba,sent_att_output = model_sent(input_tensors,label,attention_mask=sent_mask)
    pos_socre = np.exp(proba.cpu().detach().numpy())[0,1]
    y_hat.append(pos_socre)
    _,predicted = torch.max(proba,1)
    pred_list.append(predicted.item())
    y_list.append(label.item())
    correct += (predicted == label).sum()
accu = correct.item() / total_num
roc_score = sklearn.metrics.roc_auc_score(y_list, y_hat)

In [ ]:
roc_score = sklearn.metrics.roc_auc_score(y_list, y_hat)
print(roc_score)

In [ ]:
diffs = abs(np.array(y_hat) - np.array(y_list))
diff_samples = np.where(diffs>0.5)[0]

In [ ]:
optimal_point,f1_list = utils.f1_maximize(y_hat,y_list)

In [26]:
y_proba = y_hat

In [29]:
y_proba[0] = 0.008936026


In [30]:
y_hat

[0.008936026,
 0.02250839,
 0.97538924,
 0.008259573,
 0.008397106,
 0.007297954,
 0.008460839,
 0.86550313,
 0.0072808634,
 0.17914106,
 0.009298863,
 0.5269546,
 0.6166816,
 0.025314745,
 0.9664755,
 0.95886815,
 0.9487305,
 0.010107186,
 0.007063042,
 0.95596725,
 0.00763698,
 0.5083653,
 0.011274644,
 0.90672815,
 0.022633879,
 0.09693083,
 0.010551327,
 0.0069132783,
 0.008121635,
 0.97569126,
 0.020878417,
 0.9592958,
 0.008801162,
 0.9355456,
 0.97177887,
 0.116798475,
 0.42381847,
 0.011153811,
 0.013475675,
 0.01003966,
 0.012659069,
 0.0118601555,
 0.010696197,
 0.09369572,
 0.9225471,
 0.40651754,
 0.94240403,
 0.028212152,
 0.020773992,
 0.020119939,
 0.011074122,
 0.97382146,
 0.075239725,
 0.016594455,
 0.007360006,
 0.80729,
 0.01089012,
 0.13137183,
 0.0064464286,
 0.94421405,
 0.48743325,
 0.16402219,
 0.96964025,
 0.007784955,
 0.9666,
 0.009955134,
 0.012527095,
 0.009822639,
 0.06484791,
 0.40835842,
 0.39435998,
 0.30451536,
 0.95492095,
 0.943478,
 0.008746169,
 0

In [31]:
np.save('./y_hat_clinical_noN.npy',y_hat)

In [32]:
print(y_proba[0])

0.008936026


In [25]:
optimal_point,f1_list = utils.f1_maximize(y_hat[2887:],y_list[2887:])
print(optimal_point)

[[434  47]
 [ 26 215]]
{'max_f1': 0.8548707753479126, 'precision': 0.8206106870229007, 'recall': 0.8921161825726142, 'thres': 0.64}


In [23]:
optimal_point

{'max_f1': 0.9016328156113101,
 'precision': 0.8654434250764526,
 'recall': 0.940980881130507,
 'thres': 0.68}

In [36]:
statis_dict = {'false_positive':[],'false_negative':[]}
y_pred = np.array(y_hat) > 0.68
statis_dict =  utils.wrong_statis(y_list,y_pred,statis_dict)

In [41]:
statis_dict_test = {'false_positive':[],'false_negative':[]}
y_pred = np.array(y_hat) > 0.68
statis_dict_test =  utils.wrong_statis(y_list[2887:],y_pred[2887:],statis_dict_test)

In [42]:
statis_dict_test

{'false_positive': [array([  8,  21,  38,  44,  50,  60,  71,  73,  95, 112, 127, 137, 151,
         154, 163, 165, 221, 246, 270, 275, 302, 311, 329, 335, 349, 393,
         395, 449, 461, 489, 513, 522, 523, 552, 561, 592, 622, 625, 655,
         662, 666, 675, 705, 712])],
 'false_negative': [array([ 26,  27,  62,  70,  75,  88, 158, 173, 223, 226, 285, 292, 298,
         314, 385, 415, 419, 439, 444, 445, 451, 471, 475, 503, 507, 529,
         541, 562, 629, 649, 679, 699])]}

In [40]:
len(statis_dict['false_positive'][0]) + len(statis_dict['false_negative'][0])

247

In [83]:
import pickle
src_dir = '../preprocessing/result/ratio2/'
label_train = pickle.load(open(src_dir+'label_train.pkl','rb'))    
label_test = pickle.load(open(src_dir+'label_test.pkl','rb'))
label_train.extend(label_test)
y_list = label_train
y_hat = np.load('./y_hat_clinical_noN.npy')

In [84]:
roc_score = sklearn.metrics.roc_auc_score(y_list[2887:], y_hat[2887:])
print(roc_score)

0.9588340335228301


In [77]:
y_list = np.array(y_list[2887:])
y_hat  = np.array(y_hat[2887:])
diff = y_list - y_hat
obivious_wrong = np.where(diff < -0.8)[0]
print(len(obivious_wrong))

32


In [78]:
log = 'The label accroding to ICD is {}, but model predicted: {}'
for sample_index in obivious_wrong:
    print(log.format(y_list[sample_index], y_hat[sample_index]))

The label accroding to ICD is 0, but model predicted: 0.9451741576194763
The label accroding to ICD is 0, but model predicted: 0.9024368524551392
The label accroding to ICD is 0, but model predicted: 0.9695841073989868
The label accroding to ICD is 0, but model predicted: 0.9618653059005737
The label accroding to ICD is 0, but model predicted: 0.9324651956558228
The label accroding to ICD is 0, but model predicted: 0.8170803785324097
The label accroding to ICD is 0, but model predicted: 0.9519528746604919
The label accroding to ICD is 0, but model predicted: 0.8828083872795105
The label accroding to ICD is 0, but model predicted: 0.9075936675071716
The label accroding to ICD is 0, but model predicted: 0.8801155090332031
The label accroding to ICD is 0, but model predicted: 0.876316249370575
The label accroding to ICD is 0, but model predicted: 0.904143214225769
The label accroding to ICD is 0, but model predicted: 0.9385735392570496
The label accroding to ICD is 0, but model predicted:

In [79]:
obivious_wrong

array([ 38,  44,  50,  60,  71, 127, 137, 154, 163, 165, 246, 270, 302,
       311, 329, 393, 395, 449, 461, 489, 513, 522, 523, 552, 561, 592,
       622, 625, 662, 666, 705, 712])

In [60]:
obivious_wrong

array([  15,   33,  164,  174,  239,  256,  300,  326,  362,  366,  410,
        463,  712,  741,  760,  816,  843,  849,  900, 1175, 1190, 1252,
       1263, 1265, 1377, 1448, 1495, 1502, 1515, 1608, 1651, 1696, 1703,
       1747, 1756, 1800, 1867, 1909, 1988, 1996, 2025, 2112, 2139, 2233,
       2333, 2344, 2385, 2419, 2461, 2509, 2514, 2516, 2525, 2575, 2647,
       2738, 2794, 2846, 2925, 2931, 2937, 2947, 2958, 3024, 3050, 3157,
       3189, 3198, 3216, 3280, 3282, 3336, 3348, 3376, 3400, 3410, 3509,
       3512, 3549, 3553, 3592])

In [50]:
for a in obivious_wrong:
    print(a)

537
642
652
919
1348
1549
1600
1675
1699
1733
1768
1913
1991
2208
2474
2486
2709
2975
3060
3113
3172
3179
3201
3302
3331
3358
3449
3536
